# Summary Statistics Approach

In [1]:
# Local path, change this.
yellow_follow = 'C:/Users/caspe/Desktop/yellow/lib/'

import sys; sys.path.append(yellow_follow) 
import sqlite3
import pandas as pd
import ml_utils
import numpy as np

from sqlalchemy import create_engine

In [2]:
# Tensorflow
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import Huber, MSLE

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Load data

In [3]:
# Load data
folder = "C:/Users/caspe/Desktop/Paper_2_StructuralVolume/"
in_path = folder + "grid.sqlite"

db_cnx = sqlite3.connect(in_path)
df = pd.read_sql_query("SELECT * FROM 'grid';", db_cnx)

In [4]:
# Easy reference to the different features in the datasets.
s2 = [
    'b04_mean', 'b04_stdev', 'b04_min', 'b04_max',
    'b08_mean', 'b08_stdev', 'b08_min', 'b08_max',
    'b04t_mean', 'b04t_stdev', 'b04t_min', 'b04t_max',
    'b08t_mean', 'b08t_stdev', 'b08t_min', 'b08t_max',
]

bs_asc = ['bs_asc_mean', 'bs_asc_stdev', 'bs_asc_min', 'bs_asc_max']
bs_desc = ['bs_desc_mean', 'bs_desc_stdev', 'bs_desc_min', 'bs_desc_max']
coh_asc = ['coh_asc_mean', 'coh_asc_stdev', 'coh_asc_min', 'coh_asc_max']
coh_desc = ['coh_desc_mean', 'coh_desc_stdev', 'coh_desc_min', 'coh_desc_max']

nl = ['nl_mean', 'nl_stdev', 'nl_min', 'nl_max']

In [5]:
# The municipalities used as test targets
#                       Rural,                  Mix,    Dense-urban
test_municipalities = ['Lemvig', 'Silkeborg', 'Aarhus']

In [6]:
target = 'volume'
# target = 'people'
# target = 'area'

# Define the neural network

In [7]:
# Define model
def define_model(shape, name):
    model_input = Input(shape=shape, name="input")
    model = Dense(1024, activation=tfa.activations.mish, kernel_initializer="he_normal")(model_input)
    model = Dropout(0.2)(model)
    model = BatchNormalization()(model)
    model = Dense(256, activation=tfa.activations.mish, kernel_initializer="he_normal")(model)
    model = Dropout(0.2)(model)
    model = BatchNormalization()(model)
    model = Dense(64, activation=tfa.activations.mish, kernel_initializer="he_normal")(model)
    model = BatchNormalization()(model)
    model = Dense(16, activation=tfa.activations.mish, kernel_initializer="he_normal")(model)
    model = BatchNormalization()(model)

    predictions = Dense(1, activation='relu')(model)

    return Model(inputs=[model_input], outputs=predictions)

In [8]:
# Define Optimizer
def define_optimizer():
    return tfa.optimizers.Lookahead(
        Adam(
            learning_rate=tfa.optimizers.TriangularCyclicalLearningRate(
                initial_learning_rate=1e-5,
                maximal_learning_rate=1e-2,
                step_size=9,
                scale_mode='cycle',
                name='TriangularCyclicalLearningRate',
            ),
            name="Adam",
        )
    )

# Start analysis

In [9]:
analysis_to_run = [
    # nl,
    # s2,
    # bs_asc,
    # bs_desc,
    # bs_asc + bs_desc, 
    # bs_asc + coh_asc,
    # bs_desc + coh_desc,
    # bs_asc + coh_asc + s2,
    # bs_asc + coh_asc + bs_desc + coh_desc,
    bs_asc + coh_asc + bs_desc + coh_desc + s2,
    # bs_asc + coh_asc + bs_desc + coh_desc + s2 + nl,
]
analysis_name = 'bsac_bsdc_s2'

# Train and evaluate

In [10]:
all_analysis = []

for idx, analysis in enumerate(analysis_to_run):

    scores = { "analysis": str(analysis) }

    for muni in test_municipalities:
        train = df[df['muni_name'] != muni]
        test = df[df['muni_name'] == muni]

        muni_code = str(int(test['muni_code'].iloc[0]))
        
        X_train = train[analysis].values
        X_test = test[analysis].values

        y_train = train[target].values
        y_test = test[target].values

        shape = X_train.shape[1]
        model = define_model(shape, "input")

        # Compile and test model
        model.compile(
            optimizer=define_optimizer(),
            loss="mean_absolute_error",
            metrics=[
                "mean_absolute_error",
                "mean_absolute_percentage_error",
                ml_utils.median_absolute_error,
                ml_utils.median_absolute_percentage_error,
            ])
        

        plot_model(model, to_file="./summary_model.png", show_shapes=True, show_dtype=False, show_layer_names=False)

        exit()

        model.fit(
            x=X_train,
            y=y_train,
            epochs=100,
            verbose=1,
            batch_size=2048,
            validation_split=0.2,
            callbacks=[
                EarlyStopping(
                    monitor="val_loss",
                    patience=9,
                    min_delta=5.0,
                    restore_best_weights=True,
                ),
            ]
        )

        # Evaluate model
        loss, mae, mape, meae, meape = model.evaluate(X_test, y_test, verbose=1)

        scores[muni] = {
            "mean_absolute_error": mae,
            "mean_absolute_percentage_error": mape,
            "median_absolute_error": meae,
            "median_absolute_percentage_error": meape,
        }

        # Save the predicted values
        pred = model.predict(X_test)
        test[f"pred_{analysis_name}_{muni_code}"] = pred

        engine = create_engine(f"sqlite:///{folder}grid_{analysis_name}_{muni_code}.sqlite", echo=True)
        sqlite_connection = engine.connect()

        test.to_sql(f"grid_{analysis_name}_{muni_code}", sqlite_connection, if_exists='fail')
        sqlite_connection.close()


    all_analysis.append(scores)

Epoch 1/100
741/741 [==============================] - ETA: 0s - loss: 1793.1260 - mean_absolute_error: 1793.1260 - mean_absolute_percentage_error: 1028704683.9190 - median_absolute_error: 0.0303 - median_absolute_percentage_error: 3603207023.2585

KeyboardInterrupt: 

In [ ]:
for i, analysis in enumerate(all_analysis):
    print(f"Analysis: {analysis_name}")
    print("")

    combined = {
        "MAE": [],
        "MAPE": [],
        "MeAE": [],
        "MeAPE": [],
    }

    for munipality in test_municipalities:
        test_area_name = munipality
        test_data = analysis[test_area_name]

        mean_absolute_error = test_data["mean_absolute_error"]
        mean_absolute_percentage_error = test_data["mean_absolute_percentage_error"]
        median_absolute_error = test_data["median_absolute_error"]
        median_absolute_percentage_error = test_data["median_absolute_percentage_error"]

        combined["MAE"].append(mean_absolute_error)
        combined["MAPE"].append(mean_absolute_percentage_error)
        combined["MeAE"].append(median_absolute_error)
        combined["MeAPE"].append(median_absolute_percentage_error)

        print(f"    Test area: {test_area_name}")
        print(f"    Mean Absolute Error (MAE):                {ml_utils.pad(str(round(mean_absolute_error, 3)), 5, 3)}")
        print(f"    Mean Absolute Percentage Error (MAPE):    {ml_utils.pad(str(round(mean_absolute_percentage_error * 100, 3)), 5, 3)}")
        print(f"    Median Absolute Error (MeAE):             {ml_utils.pad(str(round(median_absolute_error, 3)), 5, 3)}")
        print(f"    Median Absolute Percentage Error (MeAPE): {ml_utils.pad(str(round(median_absolute_percentage_error * 100, 3)), 5, 3)}")
        print("")
    
    mae_mean = np.array(combined['MAE']).mean()
    mae_std = np.array(combined['MAE']).std()

    mape_mean = np.array(combined['MAPE']).mean()
    mape_std = np.array(combined['MAPE']).std()

    meae_mean = np.array(combined['MeAE']).mean()
    meae_std = np.array(combined['MeAE']).std()

    meape_mean = np.array(combined['MeAPE']).mean()
    meapee_std = np.array(combined['MeAPE']).std()


    print(f"    Combined Score:")
    print(f"    Mean Absolute Error (MAE):               {ml_utils.pad(str(round(mae_mean, 3)), 5, 3)} ({ml_utils.pad(str(round(mae_mean, 3)), 5, 3)} σ)")
    print(f"    Mean Absolute Percentage Error (MAPE):   {ml_utils.pad(str(round(mape_mean * 100, 3)), 5, 3)} ({ml_utils.pad(str(round(mape_std * 100, 3)), 5, 3)} σ)")
    print(f"    Median Absolute Error (MeAE):            {ml_utils.pad(str(round(meae_mean, 3)), 5, 3)} ({ml_utils.pad(str(round(meae_std, 3)), 5, 3)} σ)")
    print(f"    Median Absolute Percentage Error (MAPE): {ml_utils.pad(str(round(meape_mean * 100, 3)), 5, 3)} ({ml_utils.pad(str(round(meapee_std * 100, 3)), 5, 3)} σ)")
    print("")